# PS Parallel Programming / Sheet 11
# Fabio Valentini / MN 01018782

## Exercise 1

### Snippet a

```c
// unsigned c1;
unsigned c2 = 32 * c1;
```

- Strength Reduction

Multiplication with $2^n$ can very easily be replaced with left-shift by $n$ bits.

```c
// unsigned c1;
unsigned c2 = c1 << 5;
```

- Transformation cost

Only to be applied if one left-shift by a constant amount of bits is cheaper than one unsigned integer multiplication (should be the case on most architectures).

- Comparison with optimized assembly

GCC with the `-O3` flag applies this exact transformation, and the resulting assembly is identical.

### Snippet b

```c
// unsigned c1;
unsigned c2 = 15 * c1;
```

- Strength Reduction

The constant `15` differs by only 1 from the next power of 2 ($2^4 = 16$), and this multiplication can be replaced by a left-shift and a subtraction.

```c
// unsigned c1;
unsigned c2 = (c1 << 4) - c1;
```

- Transformation cost

This change should only be applied if the cost of one left-shift by a constant amount of bits plus the cost of an unsigned integer subtraction is less than the cost of one integer multiplication (should be the case on most architectures).

- Comparison with optimized assembly

GCC with the `-O3` flag applies this exact transformation, and the resulting assembly is identical.

### Snippet c

```c
// unsigned c1;
unsigned c2 = 96 * c1;
```

- Strength Reduction

The constant factor `96` can be split into $64 + 32 = 2^6 + 2^5$, again making it possible to replace the multiplication by two left-shifts and one addition.

```c
// unsigned c1;
unsigned c2 = (c1 << 6) + (c1 << 5);
```

Alternatively, `96` could be simplified to `(1 + 2) * 32`, also replacing the multiplication into two left-shifts and one addition.

```c
// unsigned c1;
unsigned c2 = (c1 + (c1 << 1)) << 5;
```

- Transformation cost

In both cases, one integer multiplication is replaced with two left-shifts by constant amounts plus one addition; however, the left-shift amount might make a difference here, depending on whether the target CPU supports bit-shifts by arbitrary amounts within one clock cycle. If the amount of left-shift is taken into account as a possible source of "cost", then the second snippet is to be preferred. On most architectures, two left-shifts (once by 1 bit and once by 5 bits) plus one addition should be cheaper than one integer multiplication.

- Comparison with optimized assembly

It looks like GCC 8.2.0 with the `-O3` flag prefers the second solution (at least, the produced assembly is identical when using the second snippet in *Compiler Explorer*. The first snippet produces assembly that is a few instructions longer.

### Snippet d

```c
// unsigned c1;
unsigned c2 = 0.125 * c1;
```

- Strength Reduction

Assuming that the intention is to multiply by $\frac{1}{8}$, i.e. divide by 8. This can be simplified by using a single right-shift operation by 3 bits ($2^3 = 8$), thereby eliminating any potential confusion with floating-point numbers.

```c
// unsigned c1;
unsigned c2 = c1 >> 3;
```

- Transformation cost

The cost of floating-point arithmetic is not big on modern CPUs, but one single right-shift by a constant amount of 3 bits is still probably cheaper.

- Comparison with optimized assembly

It looks like GCC 8.2.0 is confused by the floating-point number, where the intention of the programmer was probably to "just divide by 8", and the compiler produces code that converts integers to and from double-precision (?) floating point numbers, and uses double-precision floating point multiplication.

The version with the right-shift by 3 bits results in much shorter assembly and does not involve conversions between integer and floating-point numbers. However, this "compiler confusion" would also have been avoided by just writing `c1 / 8` instead (which produces the same assembly as `c1 >> 3`).

### Snippet e

```c
// unsigned *a;
unsigned sum_fifth = 0;

for (int i = 0; i < N / 5; ++i) {
    sum_fifth += a[5 * i];
}
```

- Strength Reduction

The intention seems to be to sum up every fifth element in an array of `N` unsigned integers. Instead of multiplying the index `i` in every loop iteration, the index calculation can be simplified in the loop header. This replaces one integer multiplication per iteration with one integer addition per iteration.

```c
// unsigned *a;
unsigned sum_fifth = 0;

for (int i = 0; i < N; i += 5) {
    sum_fifth += a[i];
}
```

- Transformation cost

On the level of the "high-level" language, the only change is to replace `N/5` multiplications by `N/5` additions. The transformation should only be done if multiplication is more expensive than addition (which should be the case).

- Comparison with optimized assembly

GCC 8.2.0 seems to apply the same transformation. The generated assembly for both snippets is identical.

### Snippet f

```c
// double *a;

for (int i = 0; i < N; ++i) {
    a[i] += i / 5.3;
}
```

- Strength Reduction

According to [various](https://stackoverflow.com/a/2860396) [posts](https://stackoverflow.com/a/4125074) on StackOverflow, floating point division, while typically faster than integer division, usually still takes many more clock cycles than floating point multiplication. In this case, the inverse of `5.3` can be calculated *once* outside the loop, and the division in each iteration can be replaced by a floating-point multiplication.

```c
// double *a;

double f = 1 / 5.3;

for (int i = 0; i < N; ++i) {
    a[i] += i * f;
}
```

- Transformation cost

This transformation is dependent on the fact that floating-point multiplication is faster than floating-point division, since it replaces `N` FP divisions by `N` FP multiplications plus one FP division outside the loop (which is probably done at compile time anyway).

The cost of FP multiplication and division instructions is dependent on the exact CPU hardware. For my local machine (Zen 3), both `MULPD` and `DIVPD` instructions run as a single Operation, but `MULPD` has a latency of 3 cycles, while `DIVPD` has a latency of 13.5 cycles. On other microarchitectures, the situation is similar, with division usually having more than four times the latency of a multiplication operation (Source: [Instruction cost table](https://www.agner.org/optimize/)).

- Comparison with optimized assembly

Replacing the division in the loop body by a multiplication is the only difference in the generated assembly (two `MULPD` instructions per loop iteration in my version, two `DIVPD` instructions per iteration from the original snippet). It looks like GCC 8.2.0 did not apply any transformations to this code (other than auto-vectorization) - `MULPD` and `DIVPD` are vectorized multiplication and division instructions that are present in SSE2 / AVX / AVX512 ISA extensions with different vector widths.

### Snippet g

```c
// float c1;

float c2 = -1 * c1;
```

- Strength reduction

The sign of an IEEE 754 single-precision floating-point number can be changed by flipping the sign bit (most significant bit). However, bitwise operators are not defined on floating-point numbers, so some fiddling with pointers is needed to make this work in C.

```c
// float c1;
int *c1p = (int*) &c1;
int c2i = (*c1p) ^ (1<<31);
float *c2p = (float*) &c2i;
float c2 = *c2p;
```

While this works, it is non-portable, relies on undefined behaviour, and potentially introduces additional conversions between integers and floating point numbers in the CPU.

Even replacing the multiplication by a subtraction would work in this simple case:

```c
// float c1;
float c2 = 0.0 - c1;
```

- Transformation cost

This transformation replaces one FP multiplication with a single XOR instruction (the pointer referencing and dereferencing is only necessary due to "limitations" of C and should not be reflected in the generated assembly).

In the second case, one FP multiplication is replaced by one FP subtraction.

- Comparison with optimized assembly

These "optimization" attempts were completely pointless, and even introduced additional instructions (in both cases).

GCC 8.2.0 with the `-O3` flag did the "flip most-significant bit optimization" on its own (snippet below is equivalent to `c1 XOR 0x80000000`), but with way less overhead (and it would generate the correct code for different architectures, as well, while the snippet above would break).

```
[...]
g:
    xorps   xmm0, XMMWORD PTR .LC12[rip]

[...]

.LC12:
        .long   2147483648
        .long   0
        .long   0
        .long   0
```

In this case, it looks like the compiler even generated code for a 128-bit vector instruction / register for this purpose, even though only the first 32 bits are used.

## Exercise 2

- reference for GCC command line options: <https://gcc.gnu.org/onlinedocs/gcc/Optimize-Options.html>
- job script for LCC2: `./ex2/sheet_11_2.job.sh`
- job script output: `./ex2/sheet_11_2.dat` (contains all full outputs of `perf stat -d`)

The main function and various utility functions are shared between all implementations, and only the `work` function is implemented differently for each snippet.

All functions were executed `100000` times for array size `100000`, except the "matrix multiplication" in snippet 8, which was executed only once for three `2000x2000` matrices. With these parameters, all wall times were meaningful (between a few seconds and 1-2 minutes).

### Snippet 1: Loop Unrolling

- original snippet: `./ex2/snippet1_orig.c`
- unroll factor 2: `./ex2/snippet1_unr2.c`
- unroll factor 4: `./ex2/snippet1_unr4.c`

```c
for (int i = 0; i < N - 1; ++i) {
    a[i] = b[i] + b[i + 1];
}
```

Since N is not necessarily fixed at compile time, the loop can only be partially unrolled. Since N is odd, the original snippet is equivalent to the following code:

```c
for (int i = 0; i < N - 1; i += 2) {
    a[i]     = b[i]     + b[i + 1];
    a[i + 1] = b[i + 1] + b[i + 2];
}
```

This reduces the number of jump instructions, bound checks, and counter updates by half.

```c
for (i = 0; i < N - 1; i += 4) {
    a[i]     = b[i]     + b[i + 1];
    a[i + 1] = b[i + 1] + b[i + 2];
    a[i + 2] = b[i + 2] + b[i + 3];
    a[i + 3] = b[i + 3] + b[i + 4];
}
```

For simplicity, this code snippet assumes that `N-1` is divisible by 4, but it's clear that this partial loop unrolling could be extended arbitrarily to further reduce loop overhead (at least, until the size of the unrolled loop is equal to the original `N`).

GCC even supports automatically unrolling loops with sizes that are known at compile time, with `-funroll-loops [x]` on the command line, or `#pragma GCC unroll [x]` as [source code annotation](https://gcc.gnu.org/onlinedocs/gcc/Loop-Specific-Pragmas.html), but this is not enabled by default at any optimization level.

Some interesting metrics from profiling the produced binaries on LCC2 with `perf stat -d`:

| binary              | options              | wall time | L1 cache misses | IPC  |
| ------------------- | -------------------- | --------- | --------------- | ---- |
| snippet1_orig_opto3 | `-O3`, original      |  8.977 s  | 17.35%          | 0.79 |
| snippet1_orig_unopt | `-O0`, original      | 51.237 s  |  0.04%          | 1.97 |
| snippet1_unr2_opto3 | `-O3`, unroll 2      |  7.257 s  | 60.44%          | 2.51 |
| snippet1_unr2_unopt | `-O0`, unroll 2      | 42.705 s  |  0.05%          | 2.23 |
| snippet1_unr4_opto3 | `-O3`, unroll 4      |  7.271 s  | 64.51%          | 2.09 |
| snippet1_unr4_unopt | `-O0`, unroll 4      | 38.947 s  |  0.07%          | 2.36 |
| snippet1_comp_opto3 | `-O3 -funroll-loops` |  8.339 s  |  4.13%          | 0.42 |
| snippet1_comp_unopt | `-O0 -funroll-loops` | 51.279 s  |  0.04%          | 1.97 |

So while the instructions per clock are very high with some of the binaries compiled with `-O0`, it looks like there are just way more instructions generated, since the wall time for those binaries is still high. The "best" version in both regards seems to be `snippet1_unr2_opto3` (manually unrolled two iterations, compiled with `-O3`), which is both the fastest binary and the version with the highest IPC, even though it has way higher rate of L1 cache misses than the original snipped compiled with `-O3`.

### Snippet 2: Loop Alignment (?)

- snippet: `./ex2/snippet2.c`

```c
for (int i = 1; i < N - 2; ++i) {
    b[i] = a[i - 1] + 1;
    c[i] = 2 * a[i];
    d[i] = a[i + 1] + 2;
    e[i + 1] = a[i + 2] + 3;
    f[i + 1] = e[i] + 4;
}
```

The only reference to "loop alignment" I could find was in some Stack Overflow posts and GCC documentation, where the `-falign-loops` command line option is documented. Apparently aligning the starting address of the loop in memory makes jumps faster. This can only be done manually by inserting `nop` assembly instructions, so I used the GCC feature for enabling and disabling loop alignment with compilation flags.

According to the GCC documentation, `-falign-loops` is enabled by default with `-O2` and `-O3`, but not with `-Os` (because it will make binaries slightly bigger due to padding instructions), so I added `-fno-align-loops` to the "original" optimized binary, as well.

| binary              | options                | wall time | L1 cache misses | IPC  |
| ------------------- | ---------------------- | --------- | --------------- | ---- |
| snippet2_orig_opto3 | `-O3 -fno-align-loops` | 109.353 s | 1.05%           | 0.23 |
| snippet2_orig_unopt | `-O0 -fno-align-loops` | 153.246 s | 0.03%           | 1.98 |
| snippet2_comp_opto3 | `-O3 -falign-loops`    | 110.224 s | 1.02%           | 0.23 |
| snippet2_comp_unopt | `-O0 -falign-loops`    | 152.077 s | 0.03%           | 2.00 |

Since the numbers for `-fno-align-loops` and `-falign-loops` are almost identical, either it makes no difference for my implementation of the snippet, or they are not respected by the compiler at all.

### Snippet 3: Loop-invariant Code Motion

- original snippet: `./ex2/snippet3_orig.c`
- transformed code: `./ex2/snippet3_licm.c`

```c
for (int i = 0; i < N; ++i) {
    a[i] *= hypot(0.3, 0.4);
}
```

Since `hypot(0.3, 0.4)` is independent of the loop body, it can be moved outside, where it has to be calculated only once (maybe even at compile time):

```c
double t = hypot(0.3, 0.4);
for (int i = 0; i < N; ++i) {
    a[i] *= t;
}
```

| binary              | options | wall time | L1 cache misses | IPC  |
| ------------------- | ------- | --------- | --------------- | ---- |
| snippet3_orig_opto3 | `-O3`   |  5.716 s  | 94.42%          | 2.48 |
| snippet3_orig_unopt | `-O0`   | 43.128 s  |  0.07%          | 1.69 |
| snippet3_licm_opto3 | `-O3`   |  5.720 s  | 94.43%          | 2.47 |
| snippet3_licm_unopt | `-O0`   | 42.986 s  |  0.06%          | 1.60 |

Some loop-invariant code motion functionality is enabled by `-O3`, which explains why this the transformation at the source code level has no effect in this case. However, it does not explain why the unoptimized binaries also perform almost identically - it should make a difference whether `hypot(0.3, 0.4)` is executed once per sample, or 100000 times per sample, unless GCC also performs this calculation at compile time with the `-O0` flag.

### Snippet 4: Loop Splitting

- original snippet: `./ex2/snippet4_orig.c`
- transformed code: `./ex2/snippet4_unsw.c`

```c
for (int i = 0; i < N; ++i) {
    if (N % 2) {
        a[i] = b[i] + 5;
    } else {
        a[i] = c[i] + 5;
    }
}
```

Since `N` is independent of the loop body, the modulo calculation and conditional jumps in the loop body can be avoided by splitting the loop into two loops that cover both cases (N even and N odd):

```c
if (N % 2) {
    for (int i = 0; i < N; ++i) {
        a[i] = b[i] + 5;
    }
} else {
    for (int i = 0; i < N; ++i) {
        a[i] = c[i] + 5;
    }
}
```

| binary              | options | wall time | L1 cache misses | IPC  |
| ------------------- | ------- | --------- | --------------- | ---- |
| snippet4_orig_opto3 | `-O3`   |  7.432 s  |  7.82%          | 1.23 |
| snippet4_orig_unopt | `-O0`   | 42.963 s  |  0.04%          | 1.98 |
| snippet4_unsw_opto3 | `-O3`   |  6.080 s  | 22.66%          | 1.16 |
| snippet4_unsw_unopt | `-O0`   | 39.396 s  |  0.04%          | 1.75 |

It looks like GCC 8.2.0 did not automatically do this code transformation, since the wall time is significantly improved for both unoptimized and optimized builds. Wall time for the optimized version of the transformed code runs fastest but also has the lowest instructions per clock, which probably means that is the most efficient implementation - fewest instructions to execute in the shortest time, and probably limited by memory access (highest rate of cache misses, probably because prefetching can't keep up).

### Snippet 5: Loop Distribution

- original snippet: `./ex2/snippet5_orig.c`
- transformed code: `./ex2/snippet5_dist.c`

```c
for (int i = 0; i < N; ++i) {
    sum_a += a[i];
    sum_b += b[i];
    sum_c += c[i];
}
```

Since accesses to `sum_a` and `a`, accesses to `sum_b` and `b`, and `sum_c` and `c` are independent of each other, they can be evaluated in three separate loops:

```c
// distribute loop workload
for (int i = 0; i < N; ++i) {
    sum_a += a[i];
}

// into three separate loops
for (int i = 0; i < N; ++i) {
    sum_b += b[i];
}

// for calculating three sums
for (int i = 0; i < N; ++i) {
    sum_c += c[i];
}
```

However, in this case, there is no issue with data locality, since all three arrays `a`, `b`, and `c` are read only once, so cache lines can be ejected without impact, but loop iterations (and hence bounds checks, counter updates, and conditional jumps) are tripled by splitting the loop into three loops, introducing additional overhead for no benefit.

| binary              | options | wall time | L1 cache misses | IPC  |
| ------------------- | ------- | --------- | --------------- | ---- |
| snippet5_orig_opto3 | `-O3`   |   9.339 s | 58.07%          | 0.97 |
| snippet5_orig_unopt | `-O0`   |  69.026 s |  0.34%          | 1.46 |
| snippet5_dist_opto3 | `-O3`   |  10.023 s | 49.93%          | 1.51 |
| snippet5_dist_unopt | `-O0`   | 106.944 s |  0.05%          | 1.25 |

As expected, wall time for the version with distributed loops is worse in both cases, while instructions per clock are *higher*, so there are much more instructions to execute (loop overhead) for no benefit.

### Snippet 6: Loop Fusion

- original snippet: `./ex2/snippet6_orig.c`
- transformed code: `./ex2/snippet6_fuse.c`

```c
int min = a[0];
for (int i = 1; i < N; ++i) {
    min = (a[i] < min) ? a[i] : min;
}
for (int i = 0; i < N; ++i) {
    sum += a[i];
}
```

Applying loop peeling to the second loop yields the following code, when stripping off the first iteration:

```c
int min = a[0];
for (int i = 1; i < N; ++i) {
    min = (a[i] < min) ? a[i] : min;
}
int sum = a[0];
for (int i = 1; i < N; ++i) {
    sum += a[i];
}
```

At this point, the loop headers are identical, and `min` and `sum` do not depend on each other - so the loops can be fused into one common loop, in which both `min` and `sum` are calculated:

```c
min = a[0];

// split off a[0] from sum too
sum = a[0];

// merge two loops with identical bounds
for (int i = 1; i < N; ++i) {
    min = (a[i] < min) ? a[i] : min;
    sum += a[i];
}
```

This transformation potentially reduces memory accesses to `a` by half (since each element needs to be read only once in each iteration), and also reduces loop overhead (counter updates, conditional jumps, bound checks) by half.

| binary              | options | wall time | L1 cache misses | IPC  |
| ------------------- | ------- | --------- | --------------- | ---- |
| snippet6_orig_opto3 | `-O3`   |   9.780 s | 30.15%          | 1.65 |
| snippet6_orig_unopt | `-O0`   |  73.136 s |  0.04%          | 1.38 |
| snippet6_fuse_opto3 | `-O3`   |   6.658 s | 16.44%          | 1.97 |
| snippet6_fuse_unopt | `-O0`   |  51.849 s |  0.04%          | 1.64 |

Profiling data seems to support this assumption. Wall time is shortest and instructions per clock are highest for the optimized version of the transformed code, making it the most efficient implementation.

### Snippet 7: Loop Splitting

- original snippet: `./ex2/snippet7_orig.c`
- transformed code: `./ex2/snippet7_splt.c`

```c
for (int i = 0; i < N; ++i) {
    if (i % 2) {
        a[i] = b[i] + 4;
    } else {
        a[i] = c[i] + 5;
    }
}
```

In this case, the loop body contains a modulo calculation and a conditional jump, which are both usually not cheap. Instead, this could be split into two loops, that each iterate over the odd and even elements of the array `a`, respectively, eliminating the conditional inside the loop body entirely, at the cost of iterating over the array twice.

```c
// split into two loops:
// one for even indices: !(i % 2)
for (int i = 0; i < N; i += 2) {
    a[i] = c[i] + 5;
}
// and one for odd indices: (i % 2)
for (int i = 1; i < N; i += 2) {
    a[i] = b[i] + 4;
}
```

| binary              | options | wall time | L1 cache misses | IPC  |
| ------------------- | ------- | --------- | --------------- | ---- |
| snippet7_orig_opto3 | `-O3`   |  10.367 s |  3.70%          | 3.12 |
| snippet7_orig_unopt | `-O0`   |  43.234 s |  0.09%          | 2.01 |
| snippet7_splt_opto3 | `-O3`   |   9.578 s | 98.05%          | 2.53 |
| snippet7_splt_unopt | `-O0`   |  41.017 s |  0.08%          | 1.68 |

As expected, there is little difference in wall time between fused and split loops in this case, but a much larger cache miss rate in the version with split loops (which is to be expected). However, the version with split loops seems to compile to more compact code, because IPC is lower at a slightly smaller wall time, for both optimized and unoptimized binaries.

### Snippet 8: Loop Tiling / Strip Mining

- original snippet: `./ex2/snippet8_orig.c`
- transformed code: `./ex2/snippet8_tile.c`

```c
for (int i = 0; i < N; ++i) {
    for (int j = 0; j < N; ++j) {
        for (int k = 0; k < N; ++k) {
            c[i][j] = a[i][k] * b[k][j];
        }
    }
}
```

Splitting the two-dimensional NxN arrays into blocks of size `BLOCK_SIZE` and iterating over those blocks instead of over the whole data structure at once can improve data locality:

```c
for (int ic = 0; ic < N; ic += BLOCK_SIZE) {
    for (int jc = 0; jc < N; jc += BLOCK_SIZE) {
        for (int i = ic; (i < ic + BLOCK_SIZE) && (ic < N); ++i) {
            for (int j = jc; (j < jc + BLOCK_SIZE) && (j < N); ++j) {
                for (int k = 0; k < N; ++k) {
                    c[i][j] = a[i][k] * b[k][j];
                }
            }
        }
    }
}
```

| binary              | options | wall time | L1 cache misses | IPC  |
| ------------------- | ------- | --------- | --------------- | ---- |
| snippet8_orig_opto3 | `-O3`   |  35.729 s | 68.10%          | 0.55 |
| snippet8_orig_unopt | `-O0`   |  84.521 s |  7.15%          | 1.34 |
| snippet8_tile_opto3 | `-O3`   |  20.918 s | 72.07%          | 0.93 |
| snippet8_tile_unopt | `-O0`   |  63.948 s |  7.15%          | 1.77 |

Even though the number of cache misses is about the same for both versions of the code, it looks like less time is spent waiting for memory contents in the tiled versions - since wall times are better and IPC is higher in both cases.

## Hardware + Software

- Report: Python 3.9.5, Jupyter Notebook 6.1.6